In [16]:
import os
from dotenv import load_dotenv 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    print("Cảnh báo: GOOGLE_API_KEY không được tìm thấy trong biến môi trường.")
    print("Vui lòng đảm bảo bạn đã đặt nó trong file .env hoặc thiết lập thủ công.")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Connect to Google Sheet


In [ ]:
import gspread #type:ignore
from oauth2client.service_account import ServiceAccountCredentials

# Xác thực
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/home/hai/mcp-sever/gen-lang-client-0447346298-52beea69cd6f.json", scope)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet = client.open("Daily Report").sheet1
# Lấy dữ liệu hàng đầu tiên
data = sheet.get_all_values()
print(data[0])


['Date', 'Completed', 'In Progress', 'Next Steps', 'Blockers']


# Create tool for Google Sheets

In [9]:
from langchain.tools import Tool

def read_sheet_data(query: str) -> str:
    # có thể thêm logic xử lý `query` để đọc theo yêu cầu
    values = sheet.get_all_values()
    return str(values[:5])  # trả về 5 dòng đầu

google_sheet_tool = Tool(
    name="GoogleSheetReader",
    func=read_sheet_data,
    description="Dùng để đọc dữ liệu từ Google Sheets"
)


# Run Agent

In [17]:
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

agent = initialize_agent(
    tools=[google_sheet_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("""
Viết báo cáo cho tôi lấy ngày báo cáo mới nhất trong sheet đó dựa trên format: 
    Date: 27/03/2025
    Completed:
    - [Build POC Agentic AI] Test full flow with LLM-local
    - [Build POC Agentic AI]: Run Weaviate in container, chunk btaskee documents, implement with Customer support Agent
    - [Urgent meeting] Mongodb issues
    - Support member, review code.
    In Progress:
    - [Split database] new and old BE, verify scenario
    Next Steps:
    - [Build POC Agentic AI] Optimize Prompt
    Blockers:
    - None 
""")




> Entering new AgentExecutor chain...
Okay, I need to write a report based on the most recent date in the Google Sheet. To do this, I'll first need to get the data from the Google Sheet. I'll use a query to retrieve all the data, then find the entry with the most recent date. After that, I will format the output according to the example in the prompt.
Action: GoogleSheetReader
Action Input: SELECT *
Observation: [['Date', 'Completed', 'In Progress', 'Next Steps', 'Blockers'], ['27/03/2025', '- Onboard\n- Researching about Promting, Agents\n- Prompting for Split task Agent, Booking Agent base on sources ', '- Optimize prompt slit task agent  ', '- Prompting about CS and Findtasker Agents ', 'None']]
Thought:I have the data from the Google Sheet. The first row is the header row, and the second row contains the actual data. The date is '27/03/2025'. Now I need to format the report according to the example.
Action: GoogleSheetReader
Action Input: SELECT Completed, `In Progress`, `Next St

'Date: 27/03/2025\nCompleted:\n- Onboard\n- Researching about Promting, Agents\n- Prompting for Split task Agent, Booking Agent base on sources\nIn Progress:\n- Optimize prompt slit task agent\nNext Steps:\n- Prompting about CS and Findtasker Agents\nBlockers:\n- None'